__More tutorials available on https://github.com/TuringLang/TuringTutorials__

For more information on Bayesian models and their inference methods, check **I Probabilistic Reasoning** Chapter from Mykel J. Kochenderfer, Tim A. Wheeler, And Kyle H. Wray (2022), [Algorithms for Decision Making](https://algorithmsbook.com/)

# 1. Coin flipping model 

In [ ]:
# ]add Turing

In [ ]:
using Turing, StatsPlots, Random

In [ ]:
#Model definition
@model coinflip(y) = begin
    # Prior distribution of heads.
    p ~ Beta(1, 1) #equal to U(0,1)

    N = length(y)
    for n in 1:N
        # Heads or tails are drawn from a Bernoulli distribution.
        y[n] ~ Bernoulli(p)
    end
end;

In [ ]:
# Flip coin 10 times
Random.seed!(42)
data = rand(Bernoulli(0.5), 10)

In [ ]:
# Settings for Hamiltonian Monte Carlo (HMC) sampler.
iterations = 1000
ϵ = 0.05
τ = 10

chain = sample(coinflip(data), HMC(ϵ, τ), iterations);

In [ ]:
histogram(chain[:p])

In [ ]:
# Flip coin 100 times
Random.seed!(42);
data = rand(Bernoulli(0.5), 100);

In [ ]:
chain = sample(coinflip(data), HMC(ϵ, τ), iterations);

In [ ]:
histogram(chain[:p])

# 2. Logistic regression

In [ ]:
using Distributions, RDatasets, MCMCChains, Plots, Random, Turing, StatsPlots
using StatsFuns: logistic

Random.seed!(1);

In [ ]:
# Import "Default" dataset.
data = RDatasets.dataset("ISLR", "Default");

In [ ]:
first(data,5)

In [ ]:
# Data transformation - convert Yes to 1 and No to 0
data[!,:Student] = Int.(data.Student .== "Yes")
data[!,:Default] = Int.(data.Default .== "Yes");

In [ ]:
first(data, 6)

In [ ]:
# Splitting to train and test dataset.
function split_data(df, at = 0.70)
    (r, _) = size(df)
    index = Int(round(r * at))
    train = df[1:index, :]
    test  = df[(index+1):end, :]
    return train, test
end

# Split in main dataset in 5/95 ratio.
train, test = split_data(data, 0.05);

# Creating label vectors
train_label = train[!,:Default];
test_label = test[!,:Default];

# Extracting predictors (independent variables)
train = train[!,[:Student, :Balance, :Income]];
test = test[!,[:Student, :Balance, :Income]];
train = Matrix(train);
test = Matrix(test);

In [ ]:
# Rescale data
train = (train .- mean(train, dims=1)) ./ std(train, dims=1);
test = (test .- mean(test, dims=1)) ./ std(test, dims=1);

## Model Declaration 

`logistic_regression` takes four arguments:

- `x` is our set of independent variables;
- `y` is the element we want to predict;
- `n` is the number of observations;
- `σ` is the standard deviation for priors.

Within the model, we create four coefficients (`intercept`, `student`, `balance`, and `income`) and assign a prior of normally distributed with means of zero and standard deviations of `σ`.

The `for` block creates a variable `v` which is the logistic function. We then observe the likelihood of calculating `v` given the actual label, `y[i]`.

In [ ]:
# Bayesian logistic regression (LR)
@model logistic_regression(x, y, n, σ) = begin
    intercept ~ Normal(0, σ)

    student ~ Normal(0, σ)
    balance ~ Normal(0, σ)
    income  ~ Normal(0, σ)

    for i = 1:n
        v = logistic(intercept + student*x[i, 1] + balance*x[i,2] + income*x[i,3])
        y[i] ~ Bernoulli(v)
    end
end;

In [ ]:
# Retrieve the number of observations.
n, _ = size(train)

# Sample using HMC.
chain = mapreduce(c -> sample(logistic_regression(train, train_label, n, 1), HMC(0.05, 10), 1500),
    chainscat,
    1:3
)

#Sampled parameters characteristics
describe(chain)

In [ ]:
plot(chain)

In [ ]:
l = [:student, :balance, :income]

# `Corner` function from MCMCChains shows the distributions parameters of logistic regression
corner(chain, l)

## Making Predictions

In [ ]:
function prediction(x::Matrix, chain, threshold)
    # Pull the means from each parameter's sampled values in the chain.
    intercept = mean(chain[:intercept])
    student = mean(chain[:student])
    balance = mean(chain[:balance])
    income = mean(chain[:income])

    # Retrieve the number of rows.
    n, _ = size(x)

    # Generate a vector to store our predictions.
    v = Vector{Float64}(undef, n)

    # Calculate the logistic function for each element in the test set.
    for i in 1:n
        num = logistic(intercept .+ student * x[i,1] + balance * x[i,2] + income * x[i,3])
        if num >= threshold
            v[i] = 1
        else
            v[i] = 0
        end
    end
    return v
end;

In [ ]:
# Set the prediction threshold
threshold = 0.10

# Make the predictions
predictions = prediction(test, chain, threshold)

# Calculate MSE for test set
loss = sum((predictions - test_label) .^ 2) / length(test_label)

In [ ]:
defaults = sum(test_label)
not_defaults = length(test_label) - defaults

predicted_defaults = sum(test_label .== predictions .== 1)
predicted_not_defaults = sum(test_label .== predictions .== 0)

println("Defaults: $defaults
    Predictions: $predicted_defaults
    Percentage defaults correct $(predicted_defaults/defaults)")

println("Not defaults: $not_defaults
    Predictions: $predicted_not_defaults
    Percentage non-defaults correct $(predicted_not_defaults/not_defaults)")

# 3. Bayesian Neural Networks

Bayesian Neural Network is created using a combination of Turing and [Flux](https://github.com/FluxML/Flux.jl), a suite of tools machine learning. Flux is used to specify the neural network's layers and Turing to implement the probabalistic inference, with the goal of implementing a classification algorithm.

In [ ]:
using Turing, Flux, Plots, Random

Bayesian neural network will be used to classify points in an artificial dataset. The code below generates data points arranged in a box-like pattern and displays a graph of the dataset.

In [ ]:
# Number of points to generate.
N = 100
M = round(Int, N / 4)
Random.seed!(4321)

# Generate artificial data.
x1s = rand(M) * 4.5; x2s = rand(M) * 4.5; 
xt1s = Array([[x1s[i] + 0.5; x2s[i] + 0.5] for i = 1:M])
x1s = rand(M) * 4.5; x2s = rand(M) * 4.5; 
append!(xt1s, Array([[x1s[i] - 5; x2s[i] - 5] for i = 1:M]))

x1s = rand(M) * 4.5; x2s = rand(M) * 4.5; 
xt0s = Array([[x1s[i] + 0.5; x2s[i] - 5] for i = 1:M])
x1s = rand(M) * 4.5; x2s = rand(M) * 4.5; 
append!(xt0s, Array([[x1s[i] - 5; x2s[i] + 0.5] for i = 1:M]))

# Store all the data for later.
xs = [xt1s; xt0s]
ts = [ones(2*M); zeros(2*M)]

# Plot data points.
function plot_data()
    x1 = map(e -> e[1], xt1s)
    y1 = map(e -> e[2], xt1s)
    x2 = map(e -> e[1], xt0s)
    y2 = map(e -> e[2], xt0s)

    Plots.scatter(x1,y1, color="red", clim = (0,1))
    Plots.scatter!(x2, y2, color="blue", clim = (0,1))
end

plot_data()

## Building a Neural Network with Flux

The next step is to define a feedforward neural network where parameters are expressed as distribtuions, instead of single points as with traditional neural networks. `unpack` and `nn_forward` are functions need to specify model in Turing.

`unpack` takes a vector of parameters and partitions them between weights and biases. `nn_forward` constructs a neural network with the variables generated in `unpack` and returns a prediction based on the weights provided.

The `unpack` and `nn_forward` functions are explicity designed to create a neural network with two hidden layers and one output layer.

<img width="320" alt="nn-diagram" src="https://user-images.githubusercontent.com/422990/47970321-bd172080-e038-11e8-9c6d-6c2bd790bd8a.png">

In [ ]:
# Turn a vector into a set of weights and biases.
function unpack(nn_params::AbstractVector)
    W₁ = reshape(nn_params[1:6], 3, 2);   
    b₁ = reshape(nn_params[7:9], 3)
    
    W₂ = reshape(nn_params[10:15], 2, 3); 
    b₂ = reshape(nn_params[16:17], 2)
    
    Wₒ = reshape(nn_params[18:19], 1, 2); 
    bₒ = reshape(nn_params[20:20], 1)   
    return W₁, b₁, W₂, b₂, Wₒ, bₒ
end

# Construct a neural network using Flux and return a predicted value.
function nn_forward(xs, nn_params::AbstractVector)
    W₁, b₁, W₂, b₂, Wₒ, bₒ = unpack(nn_params)
    nn = Chain(Dense(W₁, b₁, tanh),
               Dense(W₂, b₂, tanh),
               Dense(Wₒ, bₒ, σ))
    return nn(xs)
end;

The probabalistic model specification below creates a `params` variable, which has 20 normally distributed variables. Each entry in the `params` vector represents weights and biases of our neural net.

In [ ]:
# Create a regularization term and a Gaussain prior variance term.
alpha = 0.09
sig = sqrt(1.0 / alpha)

# Specify the probabalistic model.
@model bayes_nn(xs, ts) = begin
    # Create the weight and bias vector.
    nn_params ~ MvNormal(zeros(20), sig .* ones(20))
    
    # Calculate predictions for the inputs given the weights
    # and biases in theta.
    preds = nn_forward(xs, nn_params)
    
    # Observe each prediction.
    for i = 1:length(ts)
        ts[i] ~ Bernoulli(preds[i])
    end
end;

In [ ]:
# Perform inference.
N = 5000
ch = sample(bayes_nn(hcat(xs...), ts), HMC(0.05, 4),N);

## Prediction Visualization

[MAP estimation](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation) can be used to classify our population by using the set of weights that provided the highest log posterior.

In [ ]:
#Extract parameter values
theta = hcat([ch[Symbol("nn_params["*string(i)*"]")] for i in 1:20]...)

In [ ]:
# Plot the data
plot_data()

# Find the index that provided the highest log posterior in the chain.
_, i = findmax(ch[:lp])

# Extract the max row value from i.
i = i.I[1]

# Plot the posterior distribution with a contour plot.
x_range = collect(range(-6,stop=6,length=25))
y_range = collect(range(-6,stop=6,length=25))
Z = [nn_forward([x, y], theta[i, :])[1] for x=x_range, y=y_range]
contour!(x_range, y_range, Z)

In [ ]:
# Return the average predicted value across multiple weights.
function nn_predict(x, theta, num)
    mean([nn_forward(x, theta[i,:])[1] for i in 1:10:num])
end;

In [ ]:
# Plot the average prediction.
plot_data()

n_end = 1500
x_range = collect(range(-6,stop=6,length=25))
y_range = collect(range(-6,stop=6,length=25))
Z = [nn_predict([x, y], theta, n_end)[1] for x=x_range, y=y_range]
contour!(x_range, y_range, Z)